In [1]:
import geopandas as gpd
import requests
import os
import zipfile
import tempfile

## Download Vector Data
To illustrate the visualization of vector data, we download [4-digit postcodes](https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz_2056.shp.zip) from Switzerland and store them as ESRI Shapefile in the local directory `./data/`.

More information about the data can be found on the [homepage](https://www.swisstopo.admin.ch/en/official-directory-of-towns-and-cities) of Federal Office of Topography swisstopo.



In [3]:
# Download the ZIP file from the URL
url = "https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz/ortschaftenverzeichnis_plz_2056.shp.zip"
data_dir = "./data/"

os.makedirs(data_dir, exist_ok=True)

with tempfile.TemporaryDirectory() as tmpdirname:
    # Define the path for the temporary ZIP file
    temp_zip_path = os.path.join(tmpdirname, "temp.zip")

    # Download the ZIP file and write to the temporary file
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise an error for bad status codes
        with open(temp_zip_path, "wb") as f:
            f.write(response.content)
            
    # Extract the ZIP file to the specified directory
    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

## Data Preparation
We modify the data using [geopandas](https://geopandas.org/en/stable/). First, we select the required columns (ZIP4, geometry). Then, we reproject the data from the local Swiss coordinate system [LV95](https://en.wikipedia.org/wiki/Swiss_coordinate_system) to the [World Geodetic System 84](https://en.wikipedia.org/wiki/World_Geodetic_System#WGS84) and store it as [GeoJson](https://en.wikipedia.org/wiki/GeoJSON). This format is chosen to later facilitate the creation of vector tiles.

In [2]:
gdf_plz = gpd.read_file("./data/AMTOVZ_SHP_LV95/AMTOVZ_ZIP.shp")
gdf_plz = gdf_plz[["ZIP4", "geometry"]].to_crs("EPSG:4326")
gdf_plz.to_file("data/swiss_zip.geojson", driver="GeoJSON") 

## Create Vector Tileset
To create the vector tileset, we use the command line tool [tippecanoe](https://github.com/felt/tippecanoe?tab=readme-ov-file), which was initially created by Mapbox and now maintained as open source project. Briefly explaining the settings:
- `-o data/swiss_zip.pmtiles`output path of the pmtiles file. `tippecanoe` also supports other vector tile output formats like mbtiles. Find more details [here](https://github.com/felt/tippecanoe/blob/main/README.md#output-tileset).
- `--coalesce-densest-as-needed` Dynamically combine the smallest features from each zoom level into other nearby features to keep large tiles under the 500Kb size limit. Find more details [here](https://github.com/felt/tippecanoe/blob/main/README.md#dropping-a-fraction-of-features-to-keep-under-tile-size-limits).
- `-z15` sets the max zoom level for which tiles are created. `-zg` would guess the max zoom level. Find more details [here](https://github.com/felt/tippecanoe/blob/main/README.md#zoom-levels).
- `--extend-zooms-if-still-dropping` Increase the max zoom if features are still being dropped at that zoom level. Find more details [here](https://github.com/felt/tippecanoe/blob/main/README.md#zoom-levels).
- `--no-simplification-of-shared-nodes`. Shared nodes between e.g. polygons will not be simplified. Find more details [here](--no-simplification-of-shared-nodes).
- `data/swiss_zip.geojson` input path. `tippecanoe` supports `.geojson`,`.json`, `.fgb` and `.csv` input files. Find more details [here](https://github.com/felt/tippecanoe/blob/main/README.md#input-files-and-layer-names).

In [ ]:
!tippecanoe \
    -o data/swiss_zip.pmtiles \
    -z15 \
    --coalesce-densest-as-needed \
    --extend-zooms-if-still-dropping \
    --no-simplification-of-shared-nodes \
    data/swiss_zip.geojson